In [1]:
# importing libraries
import cv2
import numpy as np
import argparse
import imutils

# Create a VideoCapture object and read from input file
cap = cv2.VideoCapture('Video 1.mp4')

# Check if camera opened successfully
if (cap.isOpened()== False):
    print("Error opening video file")
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
result = cv2.VideoWriter('output_final_polka.mp4', fourcc, 20.0, (frame_width,frame_height))
# Read until video is completed
while(cap.isOpened()):
    # Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:
        #frame = cv2.resize(frame, (500, 500))
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, (36, 25, 25), (70, 255,255))
        imask = mask>0
        green = np.zeros_like(frame, np.uint8)
        green[imask] = frame[imask]
        

        gray = cv2.cvtColor(green, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY)[1]
        # find contours in the thresholded image
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        for c in cnts:
            # compute the center of the contour
            M = cv2.moments(c)
            if(M["m00"]!=0 and M["m00"]!=0):
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
                # draw the contour and center of the shape on the image
                #cv2.drawContours(frame, [c], -1, (0, 255, 0), 2)
                cv2.circle(frame, (cX, cY), 7, (0, 0, 255), -1)
                cv2.putText(frame, "center", (cX - 20, cY - 20),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        
        # Display the resulting frame
        cv2.imshow('Frame', frame)
        result.write(frame)
        # Press Q on keyboard to exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    # Break the loop
    else:
        break

# When everything done, release
# the video capture object
cap.release()
result.release()
# Closes all the frames
cv2.destroyAllWindows()
